In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [2]:
from langchain_core.tools import InjectedToolArg 
from typing import Annotated

As we are using 1st tool and getting conversion rate between two currencies. <br>

So until we don't get the conversion_rate, we can't use the 2nd tool and multiply conversion rate with base currency value. <br>

That is the reason of using InjectedToolArg it specifies that the argument is going to get injected afterwards.<br>

But we don't need to use it if the LLM is not giving suggestion for both tools, if it is giving suggestion for 1st tool only then we can append the response and send the messages again to LLM so it will directly take the actual conversion rate value from messages.


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY = os.getenv("CURRENCY_API_KEY")

In [ ]:
# Tool creation

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches a currency conversion factor between a base currency and a target currency"""

    url=f'https://v6.exchangerate-api.com/v6/{API_KEY}/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()

@tool
# def convert(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
# Not using InjectedToolArg as the model is only suggesting 1st tool so we can directly take the conversion rate after executing 1st tool
def convert(base_currency_value: float, conversion_rate: float) -> float:
    """Given a currency conversion rate this function calculates the target currency value from a given base currency value"""

    return base_currency_value * conversion_rate

In [5]:
response = get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency': 'INR'})
response

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1749168000,
 'time_last_update_utc': 'Fri, 06 Jun 2025 00:00:00 +0000',
 'time_next_update_unix': 1749254400,
 'time_next_update_utc': 'Sat, 07 Jun 2025 00:00:00 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.8661}

In [6]:
result = convert.invoke({'base_currency_value': 10, 'conversion_rate': response['conversion_rate']})
result

858.6610000000001

In [7]:
# Tool binding

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [8]:
# Tool calling

messages = [HumanMessage("What is the conversion factor between USD and INR, and based on that convert 10 usd to inr")]
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [9]:
ai_message1 = llm_with_tools.invoke(messages)
ai_message1

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--86863985-6337-4c19-9fca-e7588ff3cbbf-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '0dc1b2f8-3ec7-4ba5-bfee-b8b5d1446a22', 'type': 'tool_call'}])

In [10]:
ai_message1.tool_calls 

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '0dc1b2f8-3ec7-4ba5-bfee-b8b5d1446a22',
  'type': 'tool_call'}]

In [11]:
messages.append(ai_message1)

In [12]:
import json
# As the API returns a json-fomattted string, therefore we use json.loads() which convert it to corresponding Python object(list, int, etc)

# Execute the 1st tool and get the value of conversion rate
for tool_call in ai_message1.tool_calls:
    if tool_call['name']=='get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # fetch conversion_rate 
        conversion_rate = json.loads(tool_message1.content)['conversion_rate'] 
        # append this tool message to messages list
        messages.append(tool_message1)

Now as i got only 1 tool so now i will send the updated messages to llm again


In [13]:
print(messages)

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that convert 10 usd to inr', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--86863985-6337-4c19-9fca-e7588ff3cbbf-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '0dc1b2f8-3ec7-4ba5-bfee-b8b5d1446a22', 'type': 'tool_call'}]), ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1749168000, "time_last_update_utc": "Fri, 06 Jun 2025 00:00:00 +0000", "time_next_update_unix": 174925440

In [14]:
ai_message2 = llm_with_tools.invoke(messages)
ai_message2

AIMessage(content='The conversion factor from USD to INR is 85.8661. I can now convert 10 USD to INR.', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10.0, "conversion_rate": 85.8661}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--571be260-fb72-4617-8e55-5beaa8cb4a04-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 10.0, 'conversion_rate': 85.8661}, 'id': '572e5e03-8b47-40ff-bc61-90cffdbd11db', 'type': 'tool_call'}])

In [15]:
ai_message2.tool_calls

[{'name': 'convert',
  'args': {'base_currency_value': 10.0, 'conversion_rate': 85.8661},
  'id': '572e5e03-8b47-40ff-bc61-90cffdbd11db',
  'type': 'tool_call'}]

In [16]:
messages.append(ai_message2)

In [17]:
for tool_call in ai_message2.tool_calls:
    if tool_call['name']=='convert':
        tool_message2 = convert.invoke(tool_call)
        # append this tool message to messages list
        messages.append(tool_message2)

In [18]:
print(messages)

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that convert 10 usd to inr', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--86863985-6337-4c19-9fca-e7588ff3cbbf-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '0dc1b2f8-3ec7-4ba5-bfee-b8b5d1446a22', 'type': 'tool_call'}]), ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1749168000, "time_last_update_utc": "Fri, 06 Jun 2025 00:00:00 +0000", "time_next_update_unix": 174925440

In [19]:
final_result = llm_with_tools.invoke(messages)
final_result

AIMessage(content='The conversion factor between USD and INR is 85.8661, and 10 USD is equal to 858.661 INR.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--0009477f-e376-464a-a6b6-be2dfabb692f-0')

In [20]:
final_result.content

'The conversion factor between USD and INR is 85.8661, and 10 USD is equal to 858.661 INR.'